# `giotto-tda` persistent homology tutorial

1. Vietoris-Rips and Pipeline API quick-start,
2. Time series and model selection - gravitational waves classification using the Sliding windows Embedding, and caching for model selection.

## 1. Vietoris-Rips API

We generate a dataset of circles, spheres and toris, 10 samples for each.

In [ ]:
from data.generate_datasets import make_point_clouds
n_samples_per_class = 10
point_clouds, labels = make_point_clouds(n_samples_per_class, 10, 0.1)

print(f"(n_point_clouds, n_points, dimension) = {point_clouds.shape}")

In [ ]:
from gtda.plotting import plot_point_cloud
plot_point_cloud(point_clouds[20])

In [ ]:
from gtda.homology import VietorisRipsPersistence

VR = VietorisRipsPersistence(homology_dimensions=[0, 1, 2])

(
- import from the homology module, where many classes are available - WeightedAlpha, Cech (euclidean), Cubical...
- for VR, it is a wrapper on top of custom bindings for ripser.
- collapse_edges for reducing the size of the complex,
- execution time can be also controlled with the `max_edge_length` parameter
- Many parameters: using euclidean distance, but anything from `sklearn.pairwise` is valid; coeff : 2
)

What is unique about the `giotto-tda` API, is that we calculate the persistence diagrams for lists of point clouds.

In [ ]:
diagrams = VR.fit_transform(point_clouds)
diagrams.shape

Two observations:
1. Diagrams are padded with points on the diagonal,
2. By default, `reduced_homology==True`, so that the essential point from the diagram in dimension 0 is removed.

In [ ]:
for ind in range(3):
    VR.plot(diagrams, sample=n_samples_per_class*ind).show()

The space of multi-sets in $\mathbb{R}^2$ lacks the mathematical properties often required for statistical inference. It is customary to derive characteristics from those spaces, like "persistence entropy".

To be more precise, we view the persistence diagram as a sum of dirac measures supported on its points. The persistence entropy is the entropy of that measure.

In [ ]:
from gtda.diagrams import PersistenceEntropy
PE = PersistenceEntropy()
PE.fit_transform(diagrams[::10])

Let's use a RandomForestClassifier on those features. Similarly to `scikit-learn`, we can compose Transformers using pipelines. In `giotto-tda`, there is a dedicated function `make_pipeline`.

In [ ]:
from gtda.pipeline import make_pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

steps = [VietorisRipsPersistence(homology_dimensions=[0, 1, 2]),
         PersistenceEntropy(),
         RandomForestClassifier()]

pipeline = make_pipeline(*steps)

pcs_train, pcs_valid, labels_train, labels_valid = train_test_split(point_clouds, labels)

pipeline.fit(pcs_train, labels_train)

pipeline.score(pcs_valid, labels_valid)

## 2. Gravitational waves 

Gravitational waves (GW) are caused by many cosmic events, such as supernovaes detection, colliding black holes, or neutron stars. While challenging due to their small amplitude, identifying GW is considered as [opening a new window on the universe](https://www.ligo.caltech.edu/page/why-detect-gw). In the literature, formally, the task is often presented as a binary classification problem, which consists of deciding whether a signal contains the signature of a GW or not. Effective approaches with CNNs have been proposed, which, however, require significant data to be trained. 

In [an article](https://arxiv.org/abs/1910.08245), Chrisopher Bresten and Jae-Hun Jung proposed to include topological signatures in a CNN classifier to improve the models' performance. In this tutorial, we present an example of a classification pipeline relying only on topological features.

### 2.1 Data

We consider a setting simpler than in the article. We create a dataset as follows:

* Generate gravitational wave signals that correspond to non-spinning binary black hole mergers
* Generate a noisy time series and embed a gravitational wave signal with probability 0.5 at a random time.

The result is a set of time series of the form

$$ s = g + \epsilon \frac{1}{R}\xi $$

where $g$ is a gravitational wave signal from the reference set, $\xi$ is Gaussian noise, $\epsilon=10^{-19}$ scales the noise amplitude to the signal, and $R$ is a parameter that controls the signal-to-noise-ratio (SNR).

Here, we set $R$ to $0.65$, what gives an SNR of 17.89, placing ourselves in a favorable setting, compared to the article.

In [ ]:
from data.generate_datasets import make_gravitational_waves
from pathlib import Path

R = 0.65
n_signals = 500
DATA = Path("./data")

noisy_signals, gw_signals, labels = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, r_min=R, r_max=R, n_snr_values=5
)

print(f"Number of noisy signals: {len(noisy_signals)}")
print(f"Number of timesteps per series: {len(noisy_signals[0])}")

Next let's visualise the two different types of time series that we wish to classify: one that is pure noise vs. one that is composed of noise plus an embedded gravitational wave signal:

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# get the index corresponding to the first pure noise time series
background_idx = np.argmin(labels)
# get the index corresponding to the first noise + gravitational wave time series
signal_idx = np.argmax(labels)

ts_noise = noisy_signals[background_idx]
ts_background = noisy_signals[signal_idx]
ts_signal = gw_signals[signal_idx]

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_noise))), y=ts_noise, mode="lines", name="noise"),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(ts_background))),
        y=ts_background,
        mode="lines",
        name="background",
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_signal))), y=ts_signal, mode="lines", name="signal"),
    row=1,
    col=2,
)
fig.show()

We make two observations:
1. It is hard to distinguish the signal by eye,
2. The signal features some regularity or periodicity.

Both observations lead us to examining the _**Takens embedding**_ of the signal $s(t)$, in order to pick up the recurrent structure. Indeed, if $f$ is sampled from a dynamical system with a non-trivial recurrent structure, then, for appropriate parameters, the image by the embedding will have non-trivial topology.

More formally,, we extract a sequence of vectors in $\mathbb{R}^{d}$ of the form

$$
TD_{d,\tau} s : \mathbb{R} \to \mathbb{R}^{d}\,, \qquad t \to \begin{bmatrix}
           s(t) \\
           s(t + \tau) \\
           s(t + 2\tau) \\
           \vdots \\
           s(t + (d-1)\tau)
         \end{bmatrix},
$$
where $d$ is the embedding dimension and $\tau$ is the time delay. The quantity $(d-1)\tau$ is known as the "window size" and the difference between $t_{i+1}$ and $t_i$ is called the stride.

Let's examine what the time delay embedding of a pure gravitational wave signal looks like:

In [ ]:
from gtda.time_series import SingleTakensEmbedding
embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_gw_embedded = embedder.fit_transform(gw_signals[0])

We can use PCA to project our high-dimensional space to 3-dimensions for visualisation:

In [ ]:
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud

pca = PCA(n_components=3)
y_gw_embedded_pca = pca.fit_transform(y_gw_embedded)

plot_point_cloud(y_gw_embedded_pca)

From the plot we can see that the decaying periodic signal generated by a black hole merger emerges as a _spiral_ in the time delay embedding space! For contrast, let's compare this to one of the pure noise time series in our sample:

In [ ]:
embedding_dimension = 30
embedding_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_noise_embedded = embedder.fit_transform(noisy_signals[background_idx])

pca = PCA(n_components=3)
y_noise_embedded_pca = pca.fit_transform(y_noise_embedded)

plot_point_cloud(y_noise_embedded_pca)

Evidently, pure noise resembles a high-dimensional ball in the time delay embedding space. Let's see if we can use persistent homology to tease apart which time series contain a gravitational wave signal versus those that don't. To do so we will adapt the strategy from the original article:

## Computing and plotting of the diagrams
(for the two classes)

In [ ]:
from gtda.diagrams import PersistenceEntropy, Scaler

VR = VietorisRipsPersistence(homology_dimensions=[0, 1])
dgms = VR.fit_transform([y_gw_embedded_pca, y_noise_embedded_pca])
dgms = Scaler().fit_transform(dgms)

In [ ]:
VR.plot(dgms)

In [ ]:
VR.plot(dgms, 1)

We observe that there are slightly more, persistent 1-cycles. Let us see, experimentally, if we can find topological features which distinguish topological signals.

1. Generate 200-dimensional time delay embeddings of each time series
2. Use PCA to reduce the time delay embeddings to 3-dimensions
3. Use the Vietoris-Rips construction to calculate persistence diagrams of $H_0$ and $H_1$ generators
4. Extract feature vectors using persistence entropy
5. Train a binary classifier on the topological features

### Define the topological feature generation pipeline

We can do steps 1 and 2 by using the following ``giotto-tda`` tools:

- The ``TakensEmbedding`` transformer – instead of ``SingleTakensEmbedding`` – which will transform each time series in ``noisy_signals`` separately and return a collection of point clouds;
- ``CollectionTransformer``, which is a convenience "meta-estimator" for applying the same PCA to each point cloud resulting from step 1.

Using the ``Pipeline`` class from ``giotto-tda``, we can chain all operations up to and including step 4 as follows:

In [ ]:
from gtda.diagrams import PersistenceEntropy, Scaler
from gtda.homology import WeakAlphaPersistence
from gtda.metaestimators import CollectionTransformer
from gtda.pipeline import Pipeline
from gtda.time_series import TakensEmbedding

embedding_dimension = 200
embedding_time_delay = 10
stride = 10

embedder = TakensEmbedding(time_delay=embedding_time_delay,
                           dimension=embedding_dimension,
                           stride=stride)

batch_pca = CollectionTransformer(PCA(n_components=3), n_jobs=-1)

persistence = VietorisRipsPersistence(homology_dimensions=[0, 1], n_jobs=-1)
#persistence = WeakAlphaPersistence(homology_dimensions=[0, 1], n_jobs=-1)

scaling = Scaler() # PEntropy is invariant to scaling

entropy = PersistenceEntropy(normalize=True, nan_fill_value=-10)


steps = [("embedder", embedder),
         ("pca", batch_pca),
         ("persistence", persistence),
         ("scaling", scaling),
         ("entropy", entropy)]
topological_transformer = Pipeline(steps)

In [ ]:
features = topological_transformer.fit_transform(noisy_signals)

### Train and evaluate a model

For the final step, let's train a simple classifier on our topological features. As usual we create training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    features, labels, test_size=0.1, random_state=42
)

and then fit and evaluate our model:

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score


def print_scores(fitted_model, X_train, y_train, X_valid, y_valid):
    res = {
        "Accuracy on train:": accuracy_score(fitted_model.predict(X_train), y_train),
        "ROC AUC on train:": roc_auc_score(
            y_train, fitted_model.predict_proba(X_train)[:, 1]
        ),
        "Accuracy on valid:": accuracy_score(fitted_model.predict(X_valid), y_valid),
        "ROC AUC on valid:": roc_auc_score(
            y_valid, fitted_model.predict_proba(X_valid)[:, 1]
        ),
    }

    for k, v in res.items():
        print(f"{k}: {v:.3f}")

In [ ]:
from sklearn.linear_model import LogisticRegression
LinearSVM, RandomForest

model = LogisticRegression()
model.fit(X_train, y_train)
print_scores(model, X_train, y_train, X_valid, y_valid)

As a simple baseline, this model is not too bad - it outperforms the deep learning baseline in the article which typically fares little better than random on the raw data. However, the combination of deep learning and persistent homology is where significant performance gains are seen - we leave this as an exercise to the intrepid reader!

### Topology of gravitational waves

It has been shown that topological information contained in the Sliding window embedding of gravitational waves is relevant.

We will extend the analysis above by
- using a vectorized input, instead of the raw persistence diagram,
- interpretting the weights of the classifier, as picking out the topological features of the gravitational waves

In [ ]:
from gtda.diagrams import PersistenceImage, PersistenceLandscape
embedder = TakensEmbedding(time_delay=embedding_time_delay,
                           dimension=embedding_dimension,
                           stride=stride)

image = PersistenceImage(sigma=0.1, n_bins=10,)
landscape = PersistenceLandscape(n_layers=2, n_bins=10)

topological_image_transformer = Pipeline(steps=[*steps[0:-1], ("img", image)])
topological_landscape_transformer = Pipeline(steps=[*steps[0:-1], ("landscape", landscape)])

In [ ]:
#imgs = topological_image_transformer.fit_transform(noisy_signals)
lndscps = topological_landscape_transformer.fit_transform(noisy_signals)

In [ ]:
lndscps_flat = lndscps.reshape(lndscps.shape[0], -1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    lndscps_flat, labels, test_size=0.1, random_state=42
)

In [ ]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.svm import LinearSVC

model = LogisticRegression(penalty="l1", max_iter=2000, solver="liblinear", verbose=4)
model.fit(X_train, y_train)
print_scores(model, X_train, y_train, X_valid, y_valid)

In [ ]:
res = cross_validate(model, lndscps_flat, labels, cv=KFold(n_splits=10, shuffle=True, random_state=42), return_estimator=True)

In [ ]:
coefs = np.concatenate([est.coef_ for est in res['estimator']], axis=0)

In [ ]:
mean_coefs, std_coefs = np.mean(coefs, axis=0), np.std(coefs, axis=0)

In [ ]:
n_dim, n_layers, n_bins = len(landscape.homology_dimensions_), landscape.n_layers, image.n_bins
mean_landscape, std_landscape = [c.reshape(1, n_dim*n_layers, n_bins) for c in [mean_coefs, std_coefs]]

In [ ]:
landscape.plot(mean_lanscape)

In [ ]:
landscape.plot(std_landscape)

## Hilbert Embedding ?

In [ ]:
topological_image_transformer = Pipeline(steps=[*steps[0:-1], ("img", image)])
imgs = topological_image_transformer.fit_transform(noisy_signals)

In [ ]:
imgs_flat = imgs.reshape(imgs.shape[0], -1)
imgs.shape, imgs_flat.shape

In [ ]:
n_dim, n_bins = len(image.homology_dimensions_), image.n_bins
mean_as_img, std_as_img = [c.reshape(1, n_dim, n_bins, n_bins) for c in [mean_coefs, std_coefs]]

In [ ]:
image.plot(mean_as_img, homology_dimension_idx=1)

In [ ]:
image.plot(std_as_img, homology_dimension_idx=1)

In [ ]:
n_dim, n_bins = len(image.homology_dimensions_), image.n_bins
weights_as_img = model.coef_.reshape(1, n_dim, n_bins, n_bins)
image.plot(weights_as_img, homology_dimension_idx=1)

In [ ]:
image.plot(weights_as_img, homology_dimension_idx=0)